In [ ]:
from google.colab import files
uploaded = files.upload()

Saving cleaned_logs.csv to cleaned_logs.csv


In [ ]:
import pandas as pd

# Load cleaned access logs
access_df = pd.read_csv('cleaned_logs.csv')
access_df.head()



,log_id,employee_id,access_point,timestamp,access_type,date,hour,day_of_week,after_hours
0,1,EMP789,Storage Room,23-02-2025 17:00,authorized,23-02-2025,17,Sunday,False
1,2,EMP789,Storage Room,21-02-2025 07:00,authorized,21-02-2025,7,Friday,True
2,3,EMP123,Storage Room,23-02-2025 04:00,authorized,23-02-2025,4,Sunday,True
3,4,EMP789,Pharmacy Room,22-02-2025 11:00,authorized,22-02-2025,11,Saturday,False
4,5,EMP789,Pharmacy Room,20-02-2025 16:00,authorized,20-02-2025,16,Thursday,False


In [ ]:
from google.colab import files
uploaded = files.upload()

Saving employee_master.csv.xlsx to employee_master.csv.xlsx


In [ ]:
import pandas as pd

# Read Excel file (first sheet by default)
emp_df = pd.read_excel('employee_master.csv.xlsx')

# Preview the data
emp_df.head()


,Unnamed: 0,Unnamed: 1,Unnamed: 2,Unnamed: 3,Unnamed: 4,Unnamed: 5,Unnamed: 6
0,Employee_ID,Name,Role,Department,status,Join_Date,Termination_Date
1,EMP789,Alice,SDE2,IT,Active,2021-06-01 00:00:00,NaN
2,EMP123,Harry,HR,Executive,Active,2022-01-15 00:00:00,NaN
3,EMP456,Ben,Sales,Executive,Terminated,2020-03-20 00:00:00,2023-09-01 00:00:00
4,EMP452,Soham,SDE3,IT Manager,Active,2019-10-01 00:00:00,NaN


In [ ]:
import pandas as pd

# Read the Excel file and skip the first row (which is the fake header)
emp_df = pd.read_excel('employee_master.csv.xlsx', header=1)

# Check if columns are correct now
print(emp_df.columns)
emp_df.head()




Index(['Employee_ID', 'Name ', 'Role', 'Department', 'status', 'Join_Date',
       'Termination_Date'],
      dtype='object')


,Employee_ID,Name,Role,Department,status,Join_Date,Termination_Date
0,EMP789,Alice,SDE2,IT,Active,2021-06-01,NaT
1,EMP123,Harry,HR,Executive,Active,2022-01-15,NaT
2,EMP456,Ben,Sales,Executive,Terminated,2020-03-20,2023-09-01
3,EMP452,Soham,SDE3,IT Manager,Active,2019-10-01,NaT
4,EMP332,Ayan,Marketing,Operations,Terminated,2020-03-20,2024-02-15


In [ ]:
emp_df['Termination_Date'] = pd.to_datetime(emp_df['Termination_Date'], errors='coerce')
print(emp_df.dtypes)

Employee_ID                 object
Name                        object
Role                        object
Department                  object
status                      object
Join_Date           datetime64[ns]
Termination_Date    datetime64[ns]
dtype: object


In [ ]:
# Rename the column to match the access_df
emp_df.rename(columns={'Employee_ID': 'employee_id'}, inplace=True)
merged_df = pd.merge(access_df, emp_df, on='employee_id', how='left')


In [ ]:
merged_df['timestamp'] = pd.to_datetime(merged_df['timestamp'], errors='coerce')
print(merged_df.dtypes)


log_id                       int64
employee_id                 object
access_point                object
timestamp           datetime64[ns]
access_type                 object
date                        object
hour                         int32
day_of_week                 object
after_hours                   bool
Name                        object
Role                        object
Department                  object
status                      object
Join_Date           datetime64[ns]
Termination_Date    datetime64[ns]
dtype: object


In [ ]:
merged_df['hour'] = merged_df['timestamp'].dt.hour


In [ ]:
# Create a new column for post-termination access
merged_df['post_termination_access'] = merged_df.apply(
    lambda row: True if row['status'] == 'Terminated' and
                       pd.notnull(row['Termination_Date']) and
                       row['timestamp'] > row['Termination_Date']
                else False,
    axis=1
)

In [ ]:
violations_df = merged_df[merged_df['post_termination_access'] == True]

# Show sample violations
violations_df[['employee_id', 'timestamp', 'Termination_Date', 'status', 'post_termination_access']]


,employee_id,timestamp,Termination_Date,status,post_termination_access
5,EMP456,2025-02-21 06:00:00,2023-09-01,Terminated,True
6,EMP456,2025-02-20 03:00:00,2023-09-01,Terminated,True
9,EMP456,2025-02-15 23:00:00,2023-09-01,Terminated,True
10,EMP456,2025-02-18 22:00:00,2023-09-01,Terminated,True
11,EMP456,2025-02-20 23:00:00,2023-09-01,Terminated,True
...,...,...,...,...,...
482,EMP456,2025-02-20 08:00:00,2023-09-01,Terminated,True
484,EMP456,2025-02-22 21:00:00,2023-09-01,Terminated,True
487,EMP456,2025-02-22 09:00:00,2023-09-01,Terminated,True
493,EMP456,2025-02-16 03:00:00,2023-09-01,Terminated,True


In [ ]:
# Convert timestamp to datetime
merged_df['timestamp'] = pd.to_datetime(merged_df['timestamp'], errors='coerce')
#Extract Hour from timestamp
merged_df['hour'] = merged_df['timestamp'].dt.hour
# Flag After-Hours Access
merged_df['after_hours_access'] = merged_df['hour'].apply(lambda x: x < 8 or x > 18)
#Preview Violations
merged_df[merged_df['after_hours_access'] == True][['employee_id', 'timestamp', 'hour', 'after_hours_access']].head()




,employee_id,timestamp,hour,after_hours_access
1,EMP789,2025-02-21 07:00:00,7,True
2,EMP123,2025-02-23 04:00:00,4,True
5,EMP456,2025-02-21 06:00:00,6,True
6,EMP456,2025-02-20 03:00:00,3,True
7,EMP123,2025-02-16 23:00:00,23,True


In [ ]:
 #Count violations for each control
 df = merged_df
post_termination_count = df['post_termination_access'].sum()
after_hours_count = df['after_hours_access'].sum()
# Build summary table
summary_df = pd.DataFrame({
    'Violation Type': ['Post-Termination Access', 'After-Hours Access'],
    'Count': [post_termination_count, after_hours_count]
})

# Show summary
print(summary_df)


            Violation Type  Count
0  Post-Termination Access    166
1       After-Hours Access    253


In [ ]:
# Save the final dataset as a CSV
merged_df.to_csv('itgc_final_audit_flags.csv', index=False)

print(" File saved as 'itgc_final_audit_flags.csv'")

 File saved as 'itgc_final_audit_flags.csv'


In [ ]:
from google.colab import files
files.download('itgc_final_audit_flags.csv')


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [ ]:
merged_df['post_term_flag'] = merged_df['post_termination_access'].apply(lambda x: 1 if x else 0)
merged_df['after_hours_flag'] = merged_df['after_hours_access'].apply(lambda x: 1 if x else 0)

merged_df.to_csv('itgc_final_audit_flags.csv', index=False)


In [ ]:
from google.colab import files
files.download('itgc_final_audit_flags.csv')


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [ ]:
 #Melt flag columns into one for stacking
df_melted = merged_df.melt(
    id_vars=['employee_id'],
    value_vars=['post_term_flag', 'after_hours_flag'],
    var_name='violation_type',
    value_name='violation_flag'
)

# Keep only rows where violation_flag is 1
df_melted = df_melted[df_melted['violation_flag'] == 1]

# Save and load this CSV into Power BI
df_melted.to_csv("itgc_stacked_violation_data.csv", index=False)


In [ ]:
from google.colab import files
files.download('itgc_stacked_violation_data.csv')


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [ ]:
merged_df['employee_id'].unique()
merged_df.head(20)



,log_id,employee_id,access_point,timestamp,access_type,date,hour,day_of_week,after_hours,Name,Role,Department,status,Join_Date,Termination_Date,post_termination_access,after_hours_access,post_term_flag,after_hours_flag
0,1,EMP789,Storage Room,2025-02-23 17:00:00,authorized,23-02-2025,17,Sunday,False,Alice,SDE2,IT,Active,2021-06-01,NaT,False,False,0,0
1,2,EMP789,Storage Room,2025-02-21 07:00:00,authorized,21-02-2025,7,Friday,True,Alice,SDE2,IT,Active,2021-06-01,NaT,False,True,0,1
2,3,EMP123,Storage Room,2025-02-23 04:00:00,authorized,23-02-2025,4,Sunday,True,Harry,HR,Executive,Active,2022-01-15,NaT,False,True,0,1
3,4,EMP789,Pharmacy Room,2025-02-22 11:00:00,authorized,22-02-2025,11,Saturday,False,Alice,SDE2,IT,Active,2021-06-01,NaT,False,False,0,0
4,5,EMP789,Pharmacy Room,2025-02-20 16:00:00,authorized,20-02-2025,16,Thursday,False,Alice,SDE2,IT,Active,2021-06-01,NaT,False,False,0,0
5,6,EMP456,Storage Room,2025-02-21 06:00:00,authorized,21-02-2025,6,Friday,True,Ben,Sales,Executive,Terminated,2020-03-20,2023-09-01,True,True,1,1
6,7,EMP456,Main Entrance,2025-02-20 03:00:00,authorized,20-02-2025,3,Thursday,True,Ben,Sales,Executive,Terminated,2020-03-20,2023-09-01,True,True,1,1
7,8,EMP123,Storage Room,2025-02-16 23:00:00,authorized,16-02-2025,23,Sunday,True,Harry,HR,Executive,Active,2022-01-15,NaT,False,True,0,1
8,9,EMP789,Main Entrance,2025-02-15 09:00:00,authorized,15-02-2025,9,Saturday,False,Alice,SDE2,IT,Active,2021-06-01,NaT,False,False,0,0
9,10,EMP456,Pharmacy Room,2025-02-15 23:00:00,authorized,15-02-2025,23,Saturday,True,Ben,Sales,Executive,Terminated,2020-03-20,2023-09-01,True,True,1,1


In [ ]:
merged_df['employee_id'].unique()


array(['EMP789', 'EMP123', 'EMP456'], dtype=object)

In [ ]:
import pandas as pd

# Load both datasets
access_df = pd.read_csv('cleaned_logs.csv')
emp_df = pd.read_excel('employee_master.csv.xlsx', skiprows=1)
# Rename employee ID column for consistency
emp_df.rename(columns={'Employee_ID': 'employee_id'}, inplace=True)

# Convert both to UPPERCASE & strip extra spaces
access_df['employee_id'] = access_df['employee_id'].str.upper().str.strip()
emp_df['employee_id'] = emp_df['employee_id'].str.upper().str.strip()
emp_df['Join_Date'] = pd.to_datetime(emp_df['Join_Date'], errors='coerce')
emp_df['Termination_Date'] = pd.to_datetime(emp_df['Termination_Date'], errors='coerce')
access_df['timestamp'] = pd.to_datetime(access_df['timestamp'], errors='coerce')

/tmp/ipython-input-42-1148563926.py:14: UserWarning: Parsing dates in %d-%m-%Y %H:%M format when dayfirst=False (the default) was specified. Pass `dayfirst=True` or specify a format to silence this warning.
  access_df['timestamp'] = pd.to_datetime(access_df['timestamp'], errors='coerce')


In [ ]:
merged_df = pd.merge(access_df, emp_df, on='employee_id', how='left')
print(emp_df.columns)


Index(['employee_id', 'Name ', 'Role', 'Department', 'status', 'Join_Date',
       'Termination_Date'],
      dtype='object')


In [ ]:
# Strip whitespace from column names
emp_df.columns = emp_df.columns.str.strip()
merged_df = pd.merge(access_df, emp_df, on='employee_id', how='left')

In [ ]:
# Show all unique employee IDs in the manual employee master
print("Manual Employee Master Data (employee_id):")
print(emp_df['employee_id'].unique())

Manual Employee Master Data (employee_id):
['EMP789' 'EMP123' 'EMP456' 'EMP452' 'EMP332' 'EMP345' 'EMP786' 'EMP552'
 'EMP998' 'EMP230' 'EMP987' 'EMP876' 'EMP700']


In [ ]:
print("Kaggle Access Log Data (employee_id):")
print(access_df['employee_id'].unique())

Kaggle Access Log Data (employee_id):
['EMP789' 'EMP123' 'EMP456']


In [ ]:
#To identify employee IDs in logs but not in your employee master (i.e., missing HR records):
# Which access log IDs are NOT in employee master?
log_ids = set(access_df['employee_id'].unique())
master_ids = set(emp_df['employee_id'].unique())

unmatched_ids = log_ids - master_ids
print("Employees in access logs NOT found in master data:")
print(unmatched_ids)

Employees in access logs NOT found in master data:
set()


In [ ]:
print(merged_df[['employee_id', 'Name', 'Department']].drop_duplicates())

  employee_id   Name Department
0      EMP789  Alice         IT
2      EMP123  Harry  Executive
5      EMP456    Ben  Executive
